In [1]:
# data_ingestion.py
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# Paths for the CSV file and database
FILE_NAME = "C:/UB/Sem_1/newp/data/electricity_prices.csv"
DATABASE_NAME = "C:/UB/Sem_1/newp/data/electricity.db"

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Successfully Connected to SQLite database: {db_file}")
    except Exception as e:
        print(e)
    return conn

def load_data_to_db(csv_file, db_file):
    df = pd.read_csv(csv_file)
    engine = create_engine(f'sqlite:///{db_file}')
    df.to_sql('electricity', con=engine, if_exists='replace', index=False)
    print(f"Data loaded into database: {db_file}")
    return df

if __name__ == "__main__":
    conn = create_connection(DATABASE_NAME)
    df = load_data_to_db(FILE_NAME, DATABASE_NAME)


Successfully Connected to SQLite database: C:/UB/Sem_1/newp/data/electricity.db
Data loaded into database: C:/UB/Sem_1/newp/data/electricity.db


In [2]:
# eda.py
import os
import pandas as pd
import sqlite3
from ydata_profiling import ProfileReport

DATABASE_NAME = "C:/UB/Sem_1/newp/data/electricity.db"

def load_data_from_db(db_file):
    conn = sqlite3.connect(db_file)
    sql_query = "SELECT * FROM electricity"
    df = pd.read_sql_query(sql_query, conn)
    return df

def perform_eda(df):
    # Ensure the 'reports' directory exists
    if not os.path.exists('reports'):
        os.makedirs('reports')
    
    profile = ProfileReport(df, title="Electricity Price Data Profiling Report")
    profile.to_file("reports/electricity_price_profile.html")
    print("EDA report generated")

if __name__ == "__main__":
    df = load_data_from_db(DATABASE_NAME)
    perform_eda(df)



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Christmas Eve'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

EDA report generated


In [3]:
# model_training.py
import os
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn

DATABASE_NAME = "C:/UB/Sem_1/newp/data/electricity.db"

def load_data_from_db(db_file):
    conn = sqlite3.connect(db_file)
    sql_query = "SELECT * FROM electricity"
    df = pd.read_sql_query(sql_query, conn)
    return df

def create_preprocessor(numerical_features, categorical_features):
    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features)
        ])
    
    return preprocessor

def evaluate_models(X_train, y_train):
    models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(random_state=42),
        "Elastic Net": ElasticNet(random_state=42),
        "Gradient Boosting": GradientBoostingRegressor(random_state=42)
    }
    
    results = {}
    for name, model in models.items():
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        rmse_scores = (-scores) ** 0.5
        results[name] = rmse_scores.mean()
    
    top_models = sorted(results.items(), key=lambda x: x[1])[:3]
    return top_models

def log_baseline_model_to_mlflow(model, model_name, X_train, X_test, y_train, y_test, preprocessor):
    os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/praneeth-bojanala5/newp.mlflow"
    os.environ['MLFLOW_TRACKING_USERNAME'] = 'praneeth-bojanala5'
    os.environ['MLFLOW_TRACKING_PASSWORD'] = '15ca78d80ad1494f59c42142b650fcd36df60e03'

    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
    mlflow.set_experiment("Electricity Price Regression")

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    with mlflow.start_run():
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred, squared=False)

        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.sklearn.log_model(pipeline, "model")
        print(f"{model_name} model logged to MLflow")

if __name__ == "__main__":
    df = load_data_from_db(DATABASE_NAME)
    
    numerical_features = ['DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay',
                          'ForecastWindProduction', 'SystemLoadEA', 'SMPEA', 'ORKTemperature',
                          'ORKWindspeed', 'CO2Intensity', 'ActualWindProduction', 'SystemLoadEP2']
    categorical_features = ['Holiday']
    
    X = df.drop('SMPEP2', axis=1)
    y = df['SMPEP2']

    # Drop rows where the target variable is NaN
    X = X[~y.isna()]
    y = y.dropna()

    preprocessor = create_preprocessor(numerical_features, categorical_features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    top_models = evaluate_models(X_train, y_train)

    for model_name, _ in top_models:
        if model_name == "Linear Regression":
            model = LinearRegression()
        elif model_name == "Random Forest":
            model = RandomForestRegressor(random_state=42)
        elif model_name == "Elastic Net":
            model = ElasticNet(random_state=42)
        elif model_name == "Gradient Boosting":
            model = GradientBoostingRegressor(random_state=42)
        
        log_baseline_model_to_mlflow(model, model_name, X_train, X_test, y_train, y_test, preprocessor)


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Random Forest model logged to MLflow


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Gradient Boosting model logged to MLflow


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Linear Regression model logged to MLflow


In [48]:
# model_optimization.py
import os
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn
import numpy as np

DATABASE_NAME = "C:/UB/Sem_1/newp/data/electricity.db"

def load_data_from_db(db_file):
    conn = sqlite3.connect(db_file)
    sql_query = "SELECT * FROM electricity"
    df = pd.read_sql_query(sql_query, conn)
    return df

def create_preprocessor(numerical_features, categorical_features):
    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features)
        ])
    
    return preprocessor

def optimize_model(df, model, param_dist, model_name):
    numerical_features = ['DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay',
                          'ForecastWindProduction', 'SystemLoadEA', 'SMPEA', 'ORKTemperature',
                          'ORKWindspeed', 'CO2Intensity', 'ActualWindProduction', 'SystemLoadEP2']
    categorical_features = ['Holiday']
    
    X = df.drop('SMPEP2', axis=1)
    y = df['SMPEP2']

    # Drop rows where the target variable is NaN
    X = X[~y.isna()]
    y = y.dropna()

    preprocessor = create_preprocessor(numerical_features, categorical_features)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=10, cv=3, random_state=42, scoring='neg_mean_squared_error', n_jobs=-1)
    random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    print(f"Optimized MAE for {model_name}: {mae}")
    print(f"Optimized MSE for {model_name}: {mse}")
    print(f"Optimized RMSE for {model_name}: {rmse}")

    return best_model, mae, mse, rmse

def log_optimized_model_to_mlflow(model, model_name, mae, mse, rmse):
    os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/praneeth-bojanala5/newp.mlflow"
    os.environ['MLFLOW_TRACKING_USERNAME'] = 'praneeth-bojanala5'
    os.environ['MLFLOW_TRACKING_PASSWORD'] = '15ca78d80ad1494f59c42142b650fcd36df60e03'

    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
    mlflow.set_experiment("Electricity Price Regression")

    with mlflow.start_run():
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.sklearn.log_model(model, "model")
        print(f"{model_name} model logged to MLflow")

if __name__ == "__main__":
    df = load_data_from_db(DATABASE_NAME)
    
    # Define parameter grids for the top 3 models
    param_dist_lr = {
        'model__fit_intercept': [True, False]
    }
    param_dist_rf = {
        'model__n_estimators': [50, 100],
        'model__max_features': ['sqrt', 'log2'],
        'model__max_depth': [4, 5],
        'model__criterion': ['squared_error', 'absolute_error']
    }
    param_dist_en = {
        'model__alpha': np.logspace(-4, 0, 10),
        'model__l1_ratio': np.linspace(0, 1, 5)
    }
    param_dist_gb = {
        'model__n_estimators': [50, 100],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 4]
    }

    # Linear Regression
    model_lr, mae_lr, mse_lr, rmse_lr = optimize_model(df, LinearRegression(), param_dist_lr, "Linear Regression")
    log_optimized_model_to_mlflow(model_lr, "Linear Regression", mae_lr, mse_lr, rmse_lr)

    # Random Forest
    model_rf, mae_rf, mse_rf, rmse_rf = optimize_model(df, RandomForestRegressor(random_state=42), param_dist_rf, "Random Forest")
    log_optimized_model_to_mlflow(model_rf, "Random Forest", mae_rf, mse_rf, rmse_rf)

    # Elastic Net
    model_en, mae_en, mse_en, rmse_en = optimize_model(df, ElasticNet(random_state=42), param_dist_en, "Elastic Net")
    log_optimized_model_to_mlflow(model_en, "Elastic Net", mae_en, mse_en, rmse_en)
    
    # Gradient Boosting
    model_gb, mae_gb, mse_gb, rmse_gb = optimize_model(df, GradientBoostingRegressor(random_state=42), param_dist_gb, "Gradient Boosting")
    log_optimized_model_to_mlflow(model_gb, "Gradient Boosting", mae_gb, mse_gb, rmse_gb)


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Optimized MAE for Linear Regression: 14.221703910761253
Optimized MSE for Linear Regression: 679.1293278363725
Optimized RMSE for Linear Regression: 26.06010989685908
Linear Regression model logged to MLflow


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Optimized MAE for Random Forest: 14.194454538634176
Optimized MSE for Random Forest: 644.5406516073597
Optimized RMSE for Random Forest: 25.38780517507096
Random Forest model logged to MLflow


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Optimized MAE for Elastic Net: 14.207558707941423
Optimized MSE for Elastic Net: 678.5357676649829
Optimized RMSE for Elastic Net: 26.048719117549386
Elastic Net model logged to MLflow


c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\bunny\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Optimized MAE for Gradient Boosting: 12.650449805101104
Optimized MSE for Gradient Boosting: 567.2546689677753
Optimized RMSE for Gradient Boosting: 23.817108744929037
Gradient Boosting model logged to MLflow
